<a href="https://colab.research.google.com/github/AffanSurya/SenakApp_ML/blob/main/Model/Recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# !pip install tensorflow_recommenders

In [64]:
from typing import Dict, Text
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import random
import re
import pandas as pd
import numpy as np
import tensorflow as tf

import tensorflow_recommenders as tfrs

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
df_recipes = pd.read_csv('/content/drive/MyDrive/Dataset/recipes_reduce.csv')
df_reviews = pd.read_csv('/content/drive/MyDrive/Dataset/reviews_reduce.csv')


In [33]:
pd.set_option('display.max_columns',28)
df_reviews.head(10)

,ReviewId,Name,RecipeId,AuthorId,Rating,Review,DateSubmitted,DateModified
0,2,Jalapeno Pepper Poppers,992,2008,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,13,Kevin's Best Corned Beef,7435,1773,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
2,14,Warm Chicken A La King,44,2085,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
3,17,Spinach and Strawberry Salad,5221,2046,4,"love it, but without the bean sprouts.",2000-05-08T11:08:00Z,2000-05-08T11:08:00Z
4,19,Neiman-Marcus $250 Chocolate Chip Cookies Recipe,13307,2046,5,"chewy goodness, not crispy at all. i even thre...",2000-05-21T16:59:00Z,2000-05-21T16:59:00Z
5,24,Greek Spinach Triangles,278,1773,4,"a little greasy, but a huge hit with the guests.",2000-03-13T19:53:00Z,2000-03-13T19:53:00Z
6,26,Spinach and Strawberry Salad,5221,2369,5,I deleted the bean sprouts and replaced them w...,2000-07-18T13:36:56Z,2000-07-18T13:36:56Z
7,36,Fresh Peach Crisp,7478,2585,4,Very simple recipe to make! This is a refreshi...,2000-09-05T13:27:34Z,2000-09-05T13:27:34Z
8,46,Spinach Quiche,7226,2594,4,This recipe was really easy to follow and was ...,2000-09-12T22:49:08Z,2000-09-12T22:49:08Z
9,50,Salsa,1078,2639,5,"Your recipe was wonderful, it all fit together...",2000-09-20T14:09:06Z,2000-09-20T14:09:06Z


In [34]:
pd.set_option('display.max_columns',28)
df_recipes.head(3)

,RecipeId,Name,AuthorId,Description,Images,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,Fat,SaturatedFat,Cholesterol,Sodium,Carbohydrate,Fiber,Sugar,Protein,RecipeInstructions
0,40,Best Lemonade,1566,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,"c(""Into a 1 quart Jar with tight fitting lid, ..."
1,42,Cabbage Soup,1538,Make and share this Cabbage Soup recipe .,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,"c(""Mix everything together and bring to a boil..."
2,44,Warm Chicken A La King,1596,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",Chicken,"c(""12"", ""2"", ""3"", ""450"", ""1"", ""2"", ""1/4"", ""1"",...","c(""chicken"", ""butter"", ""flour"", ""milk"", ""celer...",5.0,23.0,895.5,66.8,31.9,405.8,557.2,29.1,3.1,5.0,45.3,"c(""Melt 1 1/2 ozs butter, add the flour and co..."


In [35]:
# train_reviews, test_reviews = train_test_split(df_reviews, test_size=0.2, random_state=42)
# train_reviews.to_csv('/content/drive/MyDrive/Dataset/train_reviews.csv', index=False)
# test_reviews.to_csv('/content/drive/MyDrive/Dataset/test_reviews.csv', index=False)

In [36]:
reviews_train = pd.read_csv("/content/drive/MyDrive/Dataset/train_reviews.csv")
reviews_test = pd.read_csv("/content/drive/MyDrive/Dataset/test_reviews.csv")

df_reviews = df_reviews.astype({'AuthorId': 'string', 'RecipeId':'string'})
reviews_train = reviews_train.astype({'AuthorId': 'string', 'RecipeId':'string'})
reviews_test = reviews_test.astype({'AuthorId': 'string', 'RecipeId':'string'})

In [37]:
uniqueUserIds = df_reviews.AuthorId.unique()
uniqueFoodIds = df_reviews.RecipeId.unique()

In [38]:
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        self.user_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=uniqueUserIds, mask_token=None),
                                        # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(uniqueUserIds)+1, embedding_dimension)
                                    ])

        self.product_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=uniqueFoodIds, mask_token=None),
                                    # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(uniqueFoodIds)+1, embedding_dimension)
                                    ])
        # Set up a retrieval task and evaluation metrics over the
        # entire dataset of candidates.
        self.ratings = tf.keras.Sequential([
                            tf.keras.layers.Dense(256, activation="relu"),
                            tf.keras.layers.Dense(64,  activation="relu"),
                            tf.keras.layers.Dense(1)
                              ])

    def call(self, userId, foodId):
        user_embeddings  = self.user_embeddings (userId)
        food_embeddings = self.product_embeddings(foodId)
        return self.ratings(tf.concat([user_embeddings, food_embeddings], axis=1))



In [39]:
# Build a model.
class FoodModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer   = tfrs.tasks.Ranking(
                                                    loss    =  tf.keras.losses.MeanSquaredError(),
                                                    metrics = [tf.keras.metrics.RootMeanSquaredError()])


    def compute_loss(self, features, training=False):
        rating_predictions = self.ranking_model(features["userID"], features["foodID"]  )

        return self.task( labels=features["rating"], predictions=rating_predictions)

In [40]:
train_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(reviews_train.AuthorId.values, tf.string),
    "foodID":tf.cast(reviews_train.RecipeId.values, tf.string),
    "rating":tf.cast(reviews_train.Rating.values, tf.float32)
})

test_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(reviews_test.AuthorId.values, tf.string),
    "foodID":tf.cast(reviews_test.RecipeId.values, tf.string),
    "rating":tf.cast(reviews_test.Rating.values, tf.float32)
})

In [41]:
tf.random.set_seed(42)

train_data = train_data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

In [42]:
model = FoodModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001))
cached_train = train_data.shuffle(100_000).batch(8192).cache()
cached_test = test_data.batch(4096).cache()
model.fit(cached_train, epochs=10)

Epoch 1/10
71/71 [==============================] - 8s 93ms/step - root_mean_squared_error: 4.4824 - loss: 20.0724 - regularization_loss: 0.0000e+00 - total_loss: 20.0724
Epoch 2/10
71/71 [==============================] - 5s 66ms/step - root_mean_squared_error: 4.2503 - loss: 18.0448 - regularization_loss: 0.0000e+00 - total_loss: 18.0448
Epoch 3/10
71/71 [==============================] - 3s 44ms/step - root_mean_squared_error: 3.9828 - loss: 15.8411 - regularization_loss: 0.0000e+00 - total_loss: 15.8411
Epoch 4/10
71/71 [==============================] - 3s 43ms/step - root_mean_squared_error: 3.6728 - loss: 13.4675 - regularization_loss: 0.0000e+00 - total_loss: 13.4675
Epoch 5/10
71/71 [==============================] - 5s 74ms/step - root_mean_squared_error: 3.3361 - loss: 11.1093 - regularization_loss: 0.0000e+00 - total_loss: 11.1093
Epoch 6/10
71/71 [==============================] - 3s 45ms/step - root_mean_squared_error: 2.9909 - loss: 8.9276 - regularization_loss: 0.0000e+

In [82]:


# Function to get the first image URL from the 'images' column
def get_image_url(recipe_df, recipe_id, image_column='Images'):
    image_urls = recipe_df[recipe_df['RecipeId'] == recipe_id][image_column].values
    if len(image_urls) > 0:
        # Using regular expression to extract the first URL
        match = re.search(r'https://.*?\.(jpg|JPG)', image_urls[0])
        if match:
            return match.group()
    return None

# Select a random user from uniqueUserIds
user_rand = random.choice(uniqueUserIds)

test_rating = {}
for m in test_data.take(10):
    test_rating[m["foodID"].numpy()] = RankingModel()(
        tf.convert_to_tensor([user_rand]), tf.convert_to_tensor([m["foodID"]])
    )

print("Top 10 recommended products for User {}: ".format(user_rand))
for m in sorted(test_rating, key=test_rating.get, reverse=True):
    recipe_id = int(m.decode())
    recipe_info = df_recipes.loc[df_recipes['RecipeId'] == recipe_id]

    # Get the recipe name and the first image URL using the function
    recipe_name = recipe_info['Name'].item()
    image_url = get_image_url(df_recipes, recipe_id)

    print("Recipe: {}, Image URL: {}".format(recipe_name, image_url))


Top 10 recommended products for User 859368: 
Recipe: Creamy Macaroni and Cheese, Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/97/09/8/r1pDwGISWWqtIWa8BGLQ_creamy-mac-and-cheese-1.jpg
Recipe: Sweet Potato Souffle, Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/14/93/28/FNTffzkoRkOTxKpoMCDz_Baked%20Sweet%20Potatoes.JPG
Recipe: Limoncello , Lemoncella, Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/13/26/4/sn9XY8qHQsayWoLvywDz_DSC05844-2.jpg
Recipe: Greek-Style Turkey Burgers, Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/13/28/5/picH7xU83.jpg
Recipe: Malva Pudding, South African Baked Dessert, Image URL: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/11/85/45/picVMuZRI.jpg
Recipe: S'more Muffins, Image URL: ht

In [62]:
print("Top 10 recommended products for User {}: ".format(user_rand))
for m in sorted(test_rating, key=test_rating.get, reverse=True):
    print(df_recipes.loc[df_recipes['RecipeId'] == int(m.decode())]['Name'].item())

Top 10 recommended products for User 1773: 
Malva Pudding, South African Baked Dessert
S'more Muffins
Reuben Casserole
Unstuffed Pepper Soup
Greek-Style Turkey Burgers
Sweet Potato Souffle
Creamy Macaroni and Cheese
Limoncello , Lemoncella
Carbonara
Karl's Favorite Cucumbers


In [45]:
export_dir = "/content/drive/MyDrive/Capstone/test_saved_model"
input_data = tf.constant(1., shape=[1, 1])
to_save = tf.function(model.ranking_model.call).get_concrete_function(
    userId=tf.TensorSpec(shape=[None], dtype=tf.string, name="userId"),
    foodId=tf.TensorSpec(shape=[None], dtype=tf.string, name="foodId")
)
tf.saved_model.save(model.ranking_model, export_dir, signatures=to_save)



# Convert the model to TF Lite.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

# Save the TF Lite model.
tflite_model_file = '/content/drive/MyDrive/Capstone/recommendationV2.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

In [46]:
# Input user id
user_id_to_recommend = "102602"

# Get the user's unique index in the vocabulary
user_index = np.where(uniqueUserIds == user_id_to_recommend)[0]

# If the user is not in the vocabulary, you can handle this case accordingly
if user_index.size == 0:
    print(f"User with id {user_id_to_recommend} not found.")
else:
    user_index = user_index[0]# Extract the scalar value
    # Generate recommendations for the specified user
    user_embeddings = model.ranking_model.user_embeddings(tf.constant([user_id_to_recommend]))
    all_food_ids = tf.constant(uniqueFoodIds, dtype=tf.string)
    food_embeddings = model.ranking_model.product_embeddings(all_food_ids)
    scores = tf.reduce_sum(user_embeddings * food_embeddings, axis=1)

    # Get the top 10 food indices based on scores
    top_food_indices = tf.argsort(scores, direction='ASCENDING')[:10]

    # Display the recommended food names
    print(f"Top 10 recommended products for User {user_id_to_recommend}:")
    for food_index in top_food_indices.numpy():
        food_name = df_recipes.loc[df_recipes['RecipeId'] == int(uniqueFoodIds[food_index])]['Name'].item()
        food_score = scores[food_index].numpy()
        print(f"{food_name}: Score = {food_score}")

Top 10 recommended products for User 102602:
Oaxacan Grilled Shrimp: Score = -0.02086794562637806
Spicy Potato-Beef Soup: Score = -0.019946152344346046
Hot Fudge Sauce--Out of This World!: Score = -0.019138241186738014
Dot's Corn Muffins: Score = -0.018940312787890434
Pumpkin Pecan Pie: Score = -0.018917039036750793
Terrific Meatloaf: Score = -0.018051382154226303
Taverns: Score = -0.017871923744678497
Supreme Garlic Mashed Potatoes: Score = -0.01745540462434292
Japanese Style Simmered Sweet Kabocha: Score = -0.017361808568239212
Twice Baked Mini Potato Dauphinoise - Potato Gratin Muffins: Score = -0.017087846994400024
